In [1]:
import pandas as pd
from copy import deepcopy
from jellyfish import jaro_winkler

In [2]:
df = pd.read_csv('psgc.csv', encoding='latin1')

df.tail()

,Region,Region Code,Province,Province Code,Municipality,Municipality Code,Barangays,Barangays Code,Population 2010
42021,REGION IV-B (MIMAROPA),170000000,ROMBLON,175900000,SANTA MARIA (IMELDA),175917000,Concepcion Norte (Pob.),175917002,2457
42022,REGION IV-B (MIMAROPA),170000000,ROMBLON,175900000,SANTA MARIA (IMELDA),175917000,Concepcion Sur,175917003,1998
42023,REGION IV-B (MIMAROPA),170000000,ROMBLON,175900000,SANTA MARIA (IMELDA),175917000,Paroyhog,175917004,647
42024,REGION IV-B (MIMAROPA),170000000,ROMBLON,175900000,SANTA MARIA (IMELDA),175917000,Santo Niño,175917005,1121
42025,REGION IV-B (MIMAROPA),170000000,ROMBLON,175900000,SANTA MARIA (IMELDA),175917000,San Isidro,175917006,501


In [3]:
# test locations
# - locations are formatted inside a list of tuples.
# - inside each tuple are two strings: the name and location and its corresponding location type.
# - there are 5 location types: 'reg', 'prov', 'mun', 'bgy', and 'none'.

# common test cases
loc1 = [('luna', 'mun'), ('isabela', 'prov')]                   # multiple tuples, correct loc_types
loc2 = [('cagayan valley', 'reg')]                              # only one tuple is given
loc3 = [('E.B. Magalona', 'mun')]                               # location spelled differently
loc4 = [('western visayas', 'prov')]                            # mismatched location types
loc5 = [('iloilo', 'none')]                                     # loc_type not given
loc6 = [('antipolo', 'none'), ('rizal', 'none')]                # more than one tuple with 'none' loc_type

loc_tests = [loc1, loc2, loc3, loc4, loc5, loc6]

In [4]:
def search(dataf, location):
    df = deepcopy(dataf)
    
    # sort list by location type from most general to most specific, w/ 'none' at the end
    loc_types = ['reg', 'prov', 'mun', 'bgy', 'none']
    loc_sorter = [loc_types.index(tup[1]) for tup in location]
    location = [x for _, x in sorted(zip(loc_sorter, location))]

    # list of row indices of possible locations
    possible_locs = [x for x in range(42026)]
    cols = ['Region', 'Province', 'Municipality', 'Barangays']
    col = 0
    
    # narrow down list of possible locations, starting from general to specific
    for tup in location:
        # first, deal with all locations that do not have a 'none' type
        if not tup[1] == 'none':
            df = df.iloc[possible_locs]
            col = 'Region'

            if tup[1] == 'prov':
                col = 'Province'
            elif tup[1] == 'mun':
                col = 'Municipality'
            elif tup[1] == 'bgy':
                col = 'Barangays'
            
            # substring search
            possible_locs = [i for i,row in df.iterrows() if tup[0].lower() in row[col].lower()]
            col = df.columns.get_loc(col)
            
        # next, deal with 'none' loc_types OR zero matches from previous search
        if tup[1] == 'none' or len(possible_locs) == 0:
            sub_possible_locs = []
            
            # search all columns for all matches per tuple
            for x in cols:
                sub_possible_locs += [i for i,row in df.iterrows() if tup[0].lower() in row[x].lower()]
            
            possible_locs = list(set(possible_locs) & set(sub_possible_locs))
    
    # still dealing with 'none' loc_types and/or zero matches from previous search
    # keep track of column where most specific area is found
    for tup in location:
        if tup[1] == 'none':
            for loc in possible_locs:
                for x in cols:
                    if (tup[0].lower() in df.iloc[loc, df.columns.get_loc(x)].lower()) and (df.columns.get_loc(x) > col):
                        col = df.columns.get_loc(x)
    
    col += 1

    # get possible area codes
    # area_codes keeps the first instance row index of each unique area code
    new_locs = []
    area_codes = []
    
    for row in possible_locs:
        if not dataf.iloc[row, col] in new_locs:
            new_locs.append(dataf.iloc[row, col])
            area_codes.append((row, dataf.iloc[row, col]))
    
    print(area_codes)
    
    # if more than one possible area, do similarity check and choose area w/ highest similarity (jaro-winkler distance)
    for tup in location:
        if len(area_codes) != 1:
            # deal first with locations that do not have a 'none' type
            if not tup[1] == 'none':
                col = df.columns.get_loc('Region')

                if tup[1] == 'prov':
                    col = df.columns.get_loc('Province')
                elif tup[1] == 'mun':
                    col = df.columns.get_loc('Municipality')
                elif tup[1] == 'bgy':
                    col = df.columns.get_loc('Barangays')
                
                jaro_distance = [jaro_winkler(tup[0].lower(), dataf.iloc[area[0], col].lower()) for area in area_codes]
        
        # return None if no area code found
        if not area_codes:
            return None
        # else, return area code
        else:
            return area_codes[0][1]
                

search(df, loc6)

[(12512, 45802002), (12513, 45802003), (12514, 45802004), (12515, 45802007), (12516, 45802008), (12517, 45802009), (12518, 45802010), (12519, 45802011), (12520, 45802012), (12521, 45802013), (11050, 43423001), (12522, 45802014), (12523, 45802015), (12524, 45802016), (12525, 45802017), (12526, 45802018), (12511, 45802001)]


45802002

In [5]:
print(df.iloc[11050, :])
print(df.iloc[12512, :])

Region               REGION IV-A (CALABARZON)
Region Code                          40000000
Province                               LAGUNA
Province Code                        43400000
Municipality                            RIZAL
Municipality Code                    43423000
Barangays                            Antipolo
Barangays Code                       43423001
Population 2010                          2396
Name: 11050, dtype: object
Region               REGION IV-A (CALABARZON)
Region Code                          40000000
Province                                RIZAL
Province Code                        45800000
Municipality                 CITY OF ANTIPOLO
Municipality Code                    45802000
Barangays                              Cupang
Barangays Code                       45802002
Population 2010                         84187
Name: 12512, dtype: object


In [6]:
# for loc in loc_tests:
#     search(df, loc)